In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import warnings
import joblib
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv('diamonds_train.csv')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


# Предварительная обработка данных

In [18]:
# Проверяем пропущенные значения
missing = df.isnull().sum()
display(missing[missing > 0] if missing.sum() > 0 else "Пропущенных значений нет")

# Проверяем категориальные переменные
categorical_cols = ['cut', 'color', 'clarity']
for col in categorical_cols:
    if col in df.columns:
        print(f"{col}: {sorted(df[col].unique())}")

# Кодируем категориальные переменные
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col])
        label_encoders[col] = le


'Пропущенных значений нет'

cut: ['Fair', 'Good', 'Ideal', 'Premium', 'Very Good']
color: ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity: ['I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2']


## Подготовка признаков

In [19]:
# Определяем признаки для обучения
feature_columns = []
for col in df.columns:
    if col != 'price' and col not in categorical_cols:
        feature_columns.append(col)

print(f"Признаки для обучения ({len(feature_columns)}): {feature_columns}")

# Подготавливаем X и y
X = df[feature_columns]
y = df['price']

print(f"Размеры: X {X.shape}, y {y.shape}")
print(f"Статистика цены - среднее: ${y.mean():,.2f}, медиана: ${y.median():,.2f}")


Признаки для обучения (9): ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_encoded', 'color_encoded', 'clarity_encoded']
Размеры: X (43018, 9), y (43018,)
Статистика цены - среднее: $3,929.50, медиана: $2,401.00


## Разделение на train/test
**ВАЖНО**: Разделение происходит ДО нормализации и стандартизации для предотвращения data leakage


Данные традиционно разделяются на три независимых набора для корректной оценки
модели. Обучающий набор используется для тренировки модели и составляет обычно
60-80% данных. Валидационный набор (10-20%) служит для настройки гиперпараметров и выбора лучшей модели. Тестовый набор (10-20%) предоставляет
беспристрастную оценку финальной производительности


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

print(f"Train: {X_train.shape[0]:,} образцов ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test: {X_test.shape[0]:,} образцов ({X_test.shape[0]/len(X)*100:.1f}%)")


Train: 34,414 образцов (80.0%)
Test: 8,604 образцов (20.0%)


## Нормализация и стандартизация данных
В реальных данных признаки имеют совершенно разные масштабы
Проблема: алгоритмы машинного обучения воспринимают большие числа как "более важные". Признак "зарплата" будет доминировать над "возрастом" просто из-за масштаба чисел.

Нормализация решает проблему разных масштабов (приводит к диапазону), а стандартизация решает проблему разных распределений (центрирует вокруг μ=0, σ=1). 

MinMaxScaler: $$ x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}} $$
StandardScaler: $$ z = \frac{(x - μ)} {σ} $$

In [21]:
# Этап 1: Нормализация (приводим к диапазону [0,1])
minmax_scaler = MinMaxScaler()
X_train_norm = minmax_scaler.fit_transform(X_train)
X_test_norm = minmax_scaler.transform(X_test)

# Этап 2: Стандартизация нормализованных данных (μ=0, σ=1)
standard_scaler = StandardScaler()
X_train_scaled = standard_scaler.fit_transform(X_train_norm)
X_test_scaled = standard_scaler.transform(X_test_norm)

## Обучение модели

In [22]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Предсказания
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)


### R² (коэффициент детерминации)

$$ R^2 = 1 - \frac{SS_{res}}{SS_{tot}} = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2} $$

где:
- $y_i$ — реальное значение
- $ \hat{y}_i $— предсказанное значение модели
- $ \bar{y} $ — среднее значение реальных данных
- $ SS_{res} $ — сумма квадратов остатков (ошибки модели)
- $ SS_{tot} $ — общая сумма квадратов (дисперсия данных)

### RMSE (Root Mean Square Error)

### Формула

$$ RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2} = \sqrt{MSE} $$

где:
- $ n $ — количество наблюдений
- $ y_i $ — реальное значение i-го наблюдения
- $ \hat{y}_i $ — предсказанное значение i-го наблюдения
- $ MSE $ — средняя квадратичная ошибка

In [23]:
# Основные метрики
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("МЕТРИКИ КАЧЕСТВА:")
print(f"R² Score - Train: {train_r2:.4f}, Test: {test_r2:.4f}")
print(f"RMSE - Train: ${train_rmse:,.2f}, Test: ${test_rmse:,.2f}")

# Кросс-валидация
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"Кросс-валидация R² (5-fold): {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

overfitting = train_r2 - test_r2
print(f"Переобучение (разница R²): {overfitting:.4f}")


МЕТРИКИ КАЧЕСТВА:
R² Score - Train: 0.8892, Test: 0.8048
RMSE - Train: $1,323.91, Test: $1,776.06
Кросс-валидация R² (5-fold): 0.8778 (+/- 0.0479)
Переобучение (разница R²): 0.0843


## Уравнение регрессии


In [24]:
print(f"Перехват: {model.intercept_:.2f}")

# Коэффициенты по важности
coef_importance = [(abs(coef), feature, coef) for feature, coef in zip(feature_columns, model.coef_)]
coef_importance.sort(reverse=True)

print("Коэффициенты по важности:")
for i, (abs_coef, feature, coef) in enumerate(coef_importance, 1):
    print(f"{i}. {feature}: {coef:+.4f}")

# Полное уравнение
equation = f"price = {model.intercept_:.2f}"
for feature, coef in zip(feature_columns, model.coef_):
    if coef >= 0:
        equation += f" + {coef:.4f}*{feature}"
    else:
        equation += f" - {abs(coef):.4f}*{feature}"

print(f"УРАВНЕНИЕ: {equation}")


Перехват: 3916.76
Коэффициенты по важности:
1. carat: +5504.2429
2. x: -3567.7978
3. y: +2050.2121
4. clarity_encoded: +483.6076
5. color_encoded: -459.0342
6. depth: -212.5148
7. table: -194.4420
8. z: -80.9027
9. cut_encoded: +56.1154
УРАВНЕНИЕ: price = 3916.76 + 5504.2429*carat - 212.5148*depth - 194.4420*table - 3567.7978*x + 2050.2121*y - 80.9027*z + 56.1154*cut_encoded - 459.0342*color_encoded + 483.6076*clarity_encoded


## Финальная модель для соревнований
Обучаем на всех данных для максимального качества


In [25]:
# Шаг 1: Нормализация
minmax_scaler = MinMaxScaler()
X_norm = minmax_scaler.fit_transform(X)

# Шаг 2: Стандартизация
standard_scaler = StandardScaler() 
X_scaled = standard_scaler.fit_transform(X_norm)

final_model = LinearRegression()
final_model.fit(X_scaled, y)


# Оценка модели
predictions = final_model.predict(X_scaled)
final_r2 = r2_score(y, predictions)
final_rmse = np.sqrt(mean_squared_error(y, predictions))

print(f"R² на всех данных: {final_r2:.4f}")
print(f"RMSE на всех данных: ${final_rmse:,.2f}")

# Сохранение всех компонентов
joblib.dump(final_model, 'final_model.pkl')
joblib.dump(minmax_scaler, 'minmax_scaler.pkl')
joblib.dump(standard_scaler, 'standard_scaler.pkl') 
joblib.dump(label_encoders, 'label_encoders.pkl')

R² на всех данных: 0.8884
RMSE на всех данных: $1,331.61


['label_encoders.pkl']

# Тестирование и сохранения файла для загрузки в соревнования

In [26]:
df_test = pd.read_csv('diamonds_test.csv')

print(df_test.head())

# Сохраняем id для submission
test_ids = df_test['id'].copy()

   id  carat        cut color clarity  depth  table     x     y     z
0   0   1.02       Good     F     SI2   59.2   58.0  6.51  6.56  3.87
1   1   0.70  Very Good     I    VVS1   59.5   58.0  5.78  5.81  3.45
2   2   0.32  Very Good     H    VVS2   63.4   56.0  4.37  4.34  2.76
3   3   0.42      Ideal     F    VVS2   62.2   56.0  4.79  4.82  2.99
4   4   0.40      Ideal     F     VS2   62.3   54.0  4.74  4.77  2.96


In [27]:
# Кодируем
for col in ['cut', 'color', 'clarity']:
    df_test[col + '_encoded'] = label_encoders[col].transform(df_test[col])

In [29]:
#  признаки
X_test = df_test[feature_columns]

# 1. нормализация (MinMaxScaler)
X_test_norm = minmax_scaler.fit_transform(X_test)
# 2. стандартизация (StandardScaler)
X_test_scaled = standard_scaler.fit_transform(X_test_norm)

# Предсказываем
predictions = final_model.predict(X_test_scaled)

# Сохраняем submission
df_result = pd.DataFrame({'id': df_test['id'], 'price': predictions})
df_result.to_csv('submission.csv', index=False)
